# LSTM/TCN lap-time model
A lightweight sequence model using the same features as `xgboost_laptime.ipynb` with an overtaking-aware simulation.


In [12]:
import os, random, math
from collections import deque
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
DEVICE = torch.device(
    "mps" if torch.backends.mps.is_available() else ("cuda" if torch.cuda.is_available() else "cpu")
)
DEVICE


device(type='mps')

In [13]:
# Load dataset
dataset_path = Path("fastf1_lap_dataset2.csv")
if not dataset_path.exists():
    dataset_path = Path("fastf1_lap_dataset.csv")

max_rows = None  # set an int to subsample for faster experiments
df = pd.read_csv(dataset_path, nrows=max_rows)
print(f"Loaded {len(df):,} rows from {dataset_path}")

df = df[df["lap_time_s"].notna()].copy()
df = df[df["lap_time_s"] > 0].copy()
print(df[["session_key", "year", "circuit_id"]].head())


Loaded 198,633 rows from fastf1_lap_dataset2.csv
                      session_key  year  circuit_id
0  2018_abu_dhabi_grand_prix_race  2018  yas_marina
4  2018_abu_dhabi_grand_prix_race  2018  yas_marina
5  2018_abu_dhabi_grand_prix_race  2018  yas_marina
6  2018_abu_dhabi_grand_prix_race  2018  yas_marina
7  2018_abu_dhabi_grand_prix_race  2018  yas_marina


In [14]:
# Feature engineering (mirror xgboost_laptime)
race_df = df[(~df["safety_car_this_lap"]) & (~df["virtual_sc_this_lap"]) & df["lap_time_s"].notna()].copy()
circuit_baseline = (
    race_df.groupby("circuit_id")["lap_time_s"].median().reset_index().rename(columns={"lap_time_s": "circuit_median_lap"})
)
df = df.merge(circuit_baseline, on="circuit_id", how="left")
df["lap_delta_s"] = df["lap_time_s"] - df["circuit_median_lap"]

# Driver skill metric per session, then averaged
session_col = "session_key"
clean = race_df.copy()
session_stats = (
    clean.groupby(session_col)["lap_time_s"].agg(session_median_lap="median", session_std_lap="std").reset_index()
)
clean = clean.merge(session_stats, on=session_col, how="left")
clean["session_std_lap"] = clean["session_std_lap"].replace(0, np.nan)
clean["session_perf_z"] = -(clean["lap_time_s"] - clean["session_median_lap"]) / clean["session_std_lap"]
clean["session_perf_z"] = clean["session_perf_z"].fillna(0.0)

driver_session_skill = clean.groupby(["driver_id", session_col])["session_perf_z"].mean().reset_index()
driver_skill_raw = driver_session_skill.groupby("driver_id")["session_perf_z"].mean()
driver_skill = (driver_skill_raw - driver_skill_raw.mean()) / driver_skill_raw.std()

df = df.merge(driver_skill.rename("driver_skill"), on="driver_id", how="left")
df["driver_skill"] = df["driver_skill"].fillna(0.0)

df["race_progress"] = df["lap_number"] / df["total_race_laps"].replace(0, np.nan).fillna(1.0)
df["laps_remaining"] = df["total_race_laps"] - df["lap_number"]

df["gap_to_ahead_s"] = df.groupby(["session_key", "driver_id"])["gap_to_ahead_s"].ffill().bfill().fillna(0.0)
df["laps_on_current_tyre"] = df["laps_on_current_tyre"].fillna(0).astype(int)
df["rainfall"] = df["rainfall"].astype(float).fillna(0.0)
df["safety_car_this_lap"] = df["safety_car_this_lap"].astype(float)

feature_cols = [
    "laps_on_current_tyre",
    "safety_car_this_lap",
    "race_progress",
    "rainfall",
    "current_position",
    "gap_to_ahead_s",
    "year",
    "driver_skill",
]
target_col = "lap_delta_s"

df_model = df[
    feature_cols
    + ["tyre_compound", "circuit_id", "driver_id", "session_key", "lap_number", "lap_delta_s", "total_race_laps"]
].copy()
df_model["tyre_compound"] = df_model["tyre_compound"].fillna("UNKNOWN")

circuit_median_map = circuit_baseline.set_index("circuit_id")["circuit_median_lap"].to_dict()
driver_skill_map = driver_skill.to_dict()

print(
    f"Prepared modeling frame: {len(df_model):,} rows, {df_model['session_key'].nunique()} sessions, "
    f"{df_model['driver_id'].nunique()} drivers."
)


Prepared modeling frame: 194,736 rows, 197 sessions, 43 drivers.


In [23]:
# Split sessions and build scalers/vocabs
seq_len = 12
batch_size = 64
num_epochs = 6
max_train_sessions = 300  # keep small for quick runs; set None for all
max_val_sessions = 80

rng = np.random.default_rng(SEED)
session_keys = df_model["session_key"].unique()
rng.shuffle(session_keys)

split_idx = int(0.8 * len(session_keys))
train_sessions = list(session_keys[:split_idx])
val_sessions = list(session_keys[split_idx:])

if max_train_sessions:
    train_sessions = train_sessions[:max_train_sessions]
if max_val_sessions:
    val_sessions = val_sessions[:max_val_sessions]

scaler = StandardScaler()
scaler.fit(df_model[df_model["session_key"].isin(train_sessions)][feature_cols])

compounds = sorted(df_model["tyre_compound"].unique().tolist())
drivers = sorted(df_model["driver_id"].unique().tolist())
circuits = sorted(df_model["circuit_id"].unique().tolist())
compound2idx = {c: i for i, c in enumerate(compounds)}
driver2idx = {d: i for i, d in enumerate(drivers)}
circuit2idx = {c: i for i, c in enumerate(circuits)}

print(f"Train sessions: {len(train_sessions)}, Val sessions: {len(val_sessions)}")
print(f"Vocab sizes -> drivers {len(drivers)}, circuits {len(circuits)}, compounds {len(compounds)}")


Train sessions: 157, Val sessions: 40
Vocab sizes -> drivers 43, circuits 35, compounds 9


In [24]:
# Dataset and dataloaders
class SequenceDataset(Dataset):
    def __init__(self, df_slice, sessions, seq_len):
        self.samples = []
        df_slice = df_slice[df_slice["session_key"].isin(sessions)].copy()
        grouped = df_slice.groupby(["session_key", "driver_id"])
        for (sess, drv), g in grouped:
            g = g.sort_values("lap_number")
            if len(g) <= seq_len:
                continue
            num = scaler.transform(g[feature_cols])
            tyre = g["tyre_compound"].map(compound2idx).fillna(0).astype(int).to_numpy()
            lap_delta = g["lap_delta_s"].to_numpy()
            circ_idx = circuit2idx.get(g["circuit_id"].iloc[0], 0)
            drv_idx = driver2idx.get(drv, 0)
            for i in range(seq_len, len(g)):
                self.samples.append(
                    (
                        num[i - seq_len : i].astype(np.float32),
                        tyre[i - seq_len : i].astype(np.int64),
                        drv_idx,
                        circ_idx,
                        float(lap_delta[i]),
                    )
                )

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]


def collate_batch(batch):
    nums, tyres, drivers_idx, circuits_idx, targets = zip(*batch)
    num_t = torch.tensor(np.stack(nums), dtype=torch.float32)
    tyre_t = torch.tensor(np.stack(tyres), dtype=torch.long)
    drv_t = torch.tensor(drivers_idx, dtype=torch.long)
    circ_t = torch.tensor(circuits_idx, dtype=torch.long)
    tgt_t = torch.tensor(targets, dtype=torch.float32)
    return num_t, tyre_t, drv_t, circ_t, tgt_t


train_ds = SequenceDataset(df_model, train_sessions, seq_len)
val_ds = SequenceDataset(df_model, val_sessions, seq_len)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=collate_batch)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=collate_batch)

len(train_ds), len(val_ds)


(117063, 32608)

In [35]:
# Model definition: small TCN block feeding an LSTM
class TCNBlock(nn.Module):
    def __init__(self, channels, kernel_size=3, dilation=1, dropout=0.1):
        super().__init__()
        padding = (kernel_size - 1) * dilation
        self.conv1 = nn.Conv1d(channels, channels, kernel_size, padding=padding, dilation=dilation)
        self.conv2 = nn.Conv1d(channels, channels, kernel_size, padding=padding, dilation=dilation)
        self.dropout = nn.Dropout(dropout)
        self.act = nn.ReLU()

    def forward(self, x):
        residual = x
        x = self.act(self.conv1(x))
        x = self.dropout(x)
        x = self.act(self.conv2(x))
        x = self.dropout(x)
        trim = x.size(-1) - residual.size(-1)
        if trim > 0:
            x = x[..., :-trim]
        return x + residual


class LapSequenceModel(nn.Module):
    def __init__(self, num_numeric, tyre_vocab, driver_vocab, circuit_vocab, emb_dim=16, tcn_channels=64, lstm_hidden=64, dropout=0.1):
        super().__init__()
        self.tyre_emb = nn.Embedding(tyre_vocab, emb_dim)
        self.driver_emb = nn.Embedding(driver_vocab, emb_dim)
        self.circuit_emb = nn.Embedding(circuit_vocab, emb_dim)

        input_dim = num_numeric + emb_dim * 3
        self.input_proj = nn.Linear(input_dim, tcn_channels)
        self.tcn1 = TCNBlock(tcn_channels, kernel_size=3, dilation=1, dropout=dropout)
        self.tcn2 = TCNBlock(tcn_channels, kernel_size=3, dilation=2, dropout=dropout)
        self.lstm = nn.LSTM(tcn_channels, lstm_hidden, num_layers=1, batch_first=True)
        self.head = nn.Sequential(
            nn.LayerNorm(lstm_hidden),
            nn.Dropout(dropout),
            nn.Linear(lstm_hidden, 1),
        )

    def forward(self, num_feats, tyre_idx, driver_idx, circuit_idx):
        tyre_e = self.tyre_emb(tyre_idx)
        drv_e = self.driver_emb(driver_idx).unsqueeze(1).expand(-1, num_feats.size(1), -1)
        circ_e = self.circuit_emb(circuit_idx).unsqueeze(1).expand(-1, num_feats.size(1), -1)

        x = torch.cat([num_feats, tyre_e, drv_e, circ_e], dim=-1)
        x = self.input_proj(x)
        x = x.transpose(1, 2)
        x = self.tcn1(x)
        x = self.tcn2(x)
        x = x.transpose(1, 2)
        lstm_out, _ = self.lstm(x)
        last = lstm_out[:, -1, :]
        return self.head(last).squeeze(-1)


model = LapSequenceModel(
    num_numeric=len(feature_cols),
    tyre_vocab=len(compounds),
    driver_vocab=len(drivers),
    circuit_vocab=len(circuits),
    emb_dim=32,
    tcn_channels=1280,
    lstm_hidden=1280,
    dropout=0.2,
).to(DEVICE)
model


LapSequenceModel(
  (tyre_emb): Embedding(9, 32)
  (driver_emb): Embedding(43, 32)
  (circuit_emb): Embedding(35, 32)
  (input_proj): Linear(in_features=104, out_features=1280, bias=True)
  (tcn1): TCNBlock(
    (conv1): Conv1d(1280, 1280, kernel_size=(3,), stride=(1,), padding=(2,))
    (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(1,), padding=(2,))
    (dropout): Dropout(p=0.2, inplace=False)
    (act): ReLU()
  )
  (tcn2): TCNBlock(
    (conv1): Conv1d(1280, 1280, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(2,))
    (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(2,))
    (dropout): Dropout(p=0.2, inplace=False)
    (act): ReLU()
  )
  (lstm): LSTM(1280, 1280, batch_first=True)
  (head): Sequential(
    (0): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): Linear(in_features=1280, out_features=1, bias=True)
  )
)

In [36]:
# Training loop
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = nn.MSELoss()

def run_epoch(loader, train: bool):
    model.train(train)
    total_loss = 0.0
    count = 0
    for num_t, tyre_t, drv_t, circ_t, tgt_t in tqdm(loader, desc="train" if train else "val", leave=False):
        num_t = num_t.to(DEVICE)
        tyre_t = tyre_t.to(DEVICE)
        drv_t = drv_t.to(DEVICE)
        circ_t = circ_t.to(DEVICE)
        tgt_t = tgt_t.to(DEVICE)

        with torch.set_grad_enabled(train):
            pred = model(num_t, tyre_t, drv_t, circ_t)
            loss = criterion(pred, tgt_t)
            if train:
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
        total_loss += loss.item() * len(tgt_t)
        count += len(tgt_t)
    return total_loss / max(count, 1)


for epoch in range(1, num_epochs + 1):
    train_loss = run_epoch(train_loader, train=True)
    val_loss = run_epoch(val_loader, train=False)
    print(f"Epoch {epoch}: train_loss={train_loss:.4f} val_loss={val_loss:.4f}")


Epoch 1: train_loss=92.3381 val_loss=56.5279


Epoch 2: train_loss=92.4715 val_loss=53.2441


Epoch 3: train_loss=91.5723 val_loss=55.4026


KeyboardInterrupt: 

In [ ]:
# Validation metrics
import math

def eval_metrics(loader):
    model.eval()
    total = 0.0
    count = 0
    with torch.no_grad():
        for num_t, tyre_t, drv_t, circ_t, tgt_t in loader:
            num_t = num_t.to(DEVICE)
            tyre_t = tyre_t.to(DEVICE)
            drv_t = drv_t.to(DEVICE)
            circ_t = circ_t.to(DEVICE)
            tgt_t = tgt_t.to(DEVICE)
            pred = model(num_t, tyre_t, drv_t, circ_t)
            diff = pred - tgt_t
            total += (diff ** 2).sum().item()
            count += tgt_t.numel()
    mse = total / max(count, 1)
    rmse = math.sqrt(mse)
    return mse, rmse

train_mse, train_rmse = eval_metrics(train_loader)
val_mse, val_rmse = eval_metrics(val_loader)
print(f"Train MSE: {train_mse:.4f}, RMSE: {train_rmse:.4f}")
print(f"Val   MSE: {val_mse:.4f}, RMSE: {val_rmse:.4f}")



Train MSE: 24.5039, RMSE: 4.9501
Val   MSE: 46.0531, RMSE: 6.7862


In [20]:
# Inference helpers and overtaking model
def format_seconds(x: float) -> str:
    minutes = int(x // 60)
    seconds = x % 60
    return f"{minutes}:{seconds:05.2f}"


def predict_delta_from_history(model, hist_num, hist_tyre, driver_idx, circuit_idx):
    model.eval()
    num_t = torch.tensor(hist_num, dtype=torch.float32, device=DEVICE).unsqueeze(0)
    tyre_t = torch.tensor(hist_tyre, dtype=torch.long, device=DEVICE).unsqueeze(0)
    drv_t = torch.tensor([driver_idx], dtype=torch.long, device=DEVICE)
    circ_t = torch.tensor([circuit_idx], dtype=torch.long, device=DEVICE)
    with torch.no_grad():
        pred = model(num_t, tyre_t, drv_t, circ_t)
    return float(pred.cpu().item())


def build_circuit_overtake_model(df, gap_threshold=1.0, min_opps_prior=200):
    tmp = df.copy()
    tmp = tmp.sort_values(["session_key", "lap_number", "current_position"])

    grp = tmp.groupby(["session_key", "driver_id"])
    tmp["prev_position"] = grp["current_position"].shift(1)
    tmp["prev_gap_to_ahead_s"] = grp["gap_to_ahead_s"].shift(1)

    valid = tmp["prev_position"].notna()
    tmp_valid = tmp[valid].copy()
    tmp_valid["positions_gained"] = (
        tmp_valid["prev_position"] - tmp_valid["current_position"]
    ).clip(lower=0)
    tmp_valid["overtake_event"] = (tmp_valid["positions_gained"] > 0).astype(int)

    overtakes_by_circuit = tmp_valid.groupby("circuit_id")["overtake_event"].sum()
    opp_mask = (
        (tmp_valid["prev_position"] > 1)
        & (tmp_valid["prev_gap_to_ahead_s"].notna())
        & (tmp_valid["prev_gap_to_ahead_s"] <= gap_threshold)
    )
    opportunities_by_circuit = tmp_valid[opp_mask].groupby("circuit_id")["driver_id"].count()

    stats = pd.concat([overtakes_by_circuit, opportunities_by_circuit], axis=1).fillna(0.0)
    stats.columns = ["overtakes", "opportunities"]
    stats["raw_rate"] = np.where(
        stats["opportunities"] > 0,
        stats["overtakes"] / stats["opportunities"],
        0.0,
    )

    global_rate = stats["raw_rate"].replace(0.0, np.nan).mean()
    if np.isnan(global_rate):
        global_rate = 0.05
    prior_w = float(min_opps_prior)
    stats["overtake_rate"] = (
        stats["raw_rate"] * stats["opportunities"] + global_rate * prior_w
    ) / (stats["opportunities"] + prior_w)

    q_low = stats["overtake_rate"].quantile(0.1)
    q_high = stats["overtake_rate"].quantile(0.9)
    if q_high <= q_low:
        stats["overtake_ease"] = 0.5
    else:
        norm = (stats["overtake_rate"] - q_low) / (q_high - q_low)
        stats["overtake_ease"] = norm.clip(0.05, 0.95)

    circuit_overtake_ease = stats["overtake_ease"].to_dict()
    return stats, circuit_overtake_ease


def overtake_success_probability(attacker_state, defender_state, circuit_id, circuit_overtake_ease, gap_start):
    ease = float(circuit_overtake_ease.get(circuit_id, 0.3))
    skill_att = float(driver_skill_map.get(attacker_state["driver_id"], 0.0))
    skill_def = float(driver_skill_map.get(defender_state["driver_id"], 0.0))
    skill_diff = skill_att - skill_def
    tyre_adv_laps = defender_state["laps_on_current_tyre"] - attacker_state["laps_on_current_tyre"]
    skill_term = 0.15 * np.tanh(skill_diff / 0.5)
    tyre_term = 0.10 * np.tanh(tyre_adv_laps / 10.0)
    gap_term = -0.15 * np.tanh(max(gap_start, 0.0) / 0.7)
    p = 0.2 + 0.6 * ease + skill_term + tyre_term + gap_term
    return float(np.clip(p, 0.01, 0.95))


def apply_overtakes_for_lap(
    circuit_id,
    drivers_by_pos,
    lap_times,
    pred_deltas,
    base_lap,
    circuit_overtake_ease,
    close_gap_threshold=1.0,
    fail_gap=0.3,
):
    lap_times = np.asarray(lap_times, dtype=float).copy()
    pred_deltas = np.asarray(pred_deltas, dtype=float).copy()
    n = len(drivers_by_pos)
    overtake_attempts = np.zeros(n, dtype=bool)
    for idx in range(1, n):
        follower = drivers_by_pos[idx]
        leader = drivers_by_pos[idx - 1]
        gap_start = float(follower["gap_to_ahead"])
        leader_time = lap_times[idx - 1]
        follower_time = lap_times[idx]
        gap_end_raw = gap_start + (follower_time - leader_time)
        going_to_pass_raw = gap_end_raw < 0.0
        close_enough = gap_start <= close_gap_threshold
        if not going_to_pass_raw and not close_enough:
            continue
        overtake_attempts[idx] = True
        margin = max(0.0, -gap_end_raw)
        base_p = 0.10 + 0.40 * min(margin / 0.5, 1.0)
        p_success = base_p * float(circuit_overtake_ease.get(circuit_id, 1.0))
        p_success = max(0.0, min(0.95, p_success))
        r = np.random.rand()
        success = (r < p_success) and going_to_pass_raw
        if success:
            continue
        desired_follower_time = leader_time + fail_gap - gap_start
        if desired_follower_time > follower_time:
            lap_times[idx] = desired_follower_time
    pred_deltas = lap_times - float(base_lap)
    return lap_times, pred_deltas, overtake_attempts


overtake_stats, circuit_overtake_ease = build_circuit_overtake_model(df)
overtake_stats.head()


,overtakes,opportunities,raw_rate,overtake_rate,overtake_ease
circuit_id,,,,,
austin,1039,1467,0.708248,0.692439,0.854548
baku,781,1336,0.584581,0.583526,0.543322
barcelona,1271,1575,0.806984,0.781012,0.950000
budapest,1066,1737,0.613702,0.609859,0.618569
hockenheim,364,434,0.838710,0.755987,0.950000


In [21]:
# Race simulation using the LSTM/TCN model
def build_feature_vector(state, lap, total_laps, safety_car_laps, rain_laps):
    row = {
        "laps_on_current_tyre": state["laps_on_current_tyre"] + 1,
        "safety_car_this_lap": 1.0 if lap in safety_car_laps else 0.0,
        "race_progress": lap / total_laps,
        "rainfall": 1.0 if lap in rain_laps else 0.0,
        "current_position": state["position"],
        "gap_to_ahead_s": state.get("gap_to_ahead", 0.0),
        "year": state["year"],
        "driver_skill": driver_skill_map.get(state["driver_id"], 0.0),
    }
    num = scaler.transform(pd.DataFrame([row], columns=feature_cols))[0].astype(np.float32)
    tyre_idx = compound2idx.get(state["tyre_compound"], 0)
    return num, tyre_idx


def simulate_race(
    model,
    circuit_id,
    grid_drivers,
    total_laps=50,
    year=2025,
    global_strategy=None,
    driver_strategies=None,
    safety_car_laps=None,
    rain_laps=None,
    pit_loss=20.0,
):
    if global_strategy is None:
        global_strategy = [(int(total_laps * 0.4), "MEDIUM"), (int(total_laps * 0.7), "SOFT")]
    if driver_strategies is None:
        driver_strategies = {}
    safety_car_laps = set(safety_car_laps or [])
    rain_laps = set(rain_laps or [])

    base_lap = circuit_median_map.get(circuit_id)
    if base_lap is None:
        raise ValueError(f"No circuit_median_lap for circuit_id={circuit_id}")
    circuit_idx = circuit2idx.get(circuit_id, 0)

    grid_pos_map = {drv: idx + 1 for idx, drv in enumerate(grid_drivers)}

    driver_states = []
    for idx, drv in enumerate(grid_drivers):
        strat = driver_strategies.get(drv, global_strategy)
        stops_map = {int(lap): compound for lap, compound in strat}
        state = {
            "driver_id": drv,
            "driver_idx": driver2idx.get(drv, 0),
            "circuit_idx": circuit_idx,
            "grid_position": idx + 1,
            "position": idx + 1,
            "cumul_time": float(idx * 0.3),
            "laps_on_current_tyre": 1,
            "tyre_compound": "SOFT",
            "gap_to_ahead": 0.0,
            "stops": stops_map,
            "history": [],
            "year": year,
            "feature_history_num": deque(maxlen=seq_len),
            "feature_history_tyre": deque(maxlen=seq_len),
        }
        num_vec, tyre_idx = build_feature_vector(state, lap=1, total_laps=total_laps, safety_car_laps=safety_car_laps, rain_laps=rain_laps)
        for _ in range(seq_len):
            state["feature_history_num"].append(num_vec)
            state["feature_history_tyre"].append(tyre_idx)
        driver_states.append(state)

    race_log = []

    for lap in range(1, total_laps + 1):
        prev_positions = {s["driver_id"]: s["position"] for s in driver_states}
        drivers_by_pos = sorted(driver_states, key=lambda s: s["position"])

        for idx, s in enumerate(drivers_by_pos):
            if idx == 0:
                s["gap_to_ahead"] = 0.0
            else:
                ahead = drivers_by_pos[idx - 1]
                s["gap_to_ahead"] = s["cumul_time"] - ahead["cumul_time"]

        lap_times = []
        pred_deltas = []
        for s in drivers_by_pos:
            num_vec, tyre_idx = build_feature_vector(
                s, lap=lap, total_laps=total_laps, safety_car_laps=safety_car_laps, rain_laps=rain_laps
            )
            s["feature_history_num"].append(num_vec)
            s["feature_history_tyre"].append(tyre_idx)
            hist_num = np.stack(s["feature_history_num"])[-seq_len:]
            hist_tyre = np.array(s["feature_history_tyre"])[-seq_len:]
            delta = predict_delta_from_history(model, hist_num, hist_tyre, s["driver_idx"], s["circuit_idx"])
            lap_time = base_lap + delta
            lap_times.append(lap_time)
            pred_deltas.append(delta)

        lap_times, pred_deltas, overtake_attempts = apply_overtakes_for_lap(
            circuit_id=circuit_id,
            drivers_by_pos=drivers_by_pos,
            lap_times=lap_times,
            pred_deltas=pred_deltas,
            base_lap=base_lap,
            circuit_overtake_ease=circuit_overtake_ease,
            close_gap_threshold=1.0,
            fail_gap=0.3,
        )

        attempts_this_lap = {drivers_by_pos[i]["driver_id"]: bool(overtake_attempts[i]) for i in range(len(drivers_by_pos))}

        for idx, s in enumerate(drivers_by_pos):
            lap_time = float(lap_times[idx])
            delta = float(pred_deltas[idx])
            laps_on_current_tyre_next = s["laps_on_current_tyre"] + 1
            compound_this_lap = s["tyre_compound"]
            pit_compound = s["stops"].get(lap)
            pitted = False
            if pit_compound is not None:
                lap_time += pit_loss
                pitted = True

            s["laps_on_current_tyre"] = laps_on_current_tyre_next
            s["cumul_time"] += lap_time
            s["history"].append(
                {
                    "lap": lap,
                    "lap_time": lap_time,
                    "delta": delta,
                    "tyre_compound": compound_this_lap,
                    "pitted": pitted,
                    "overtake_attempt": attempts_this_lap.get(s["driver_id"], False),
                }
            )

            if pit_compound is not None:
                s["tyre_compound"] = pit_compound
                s["laps_on_current_tyre"] = 1

        driver_states = sorted(driver_states, key=lambda s: (s["cumul_time"], s["grid_position"]))
        for pos, s in enumerate(driver_states, start=1):
            s["position"] = pos

        leader_time = driver_states[0]["cumul_time"]
        for s in driver_states:
            last_lap = s["history"][-1]
            gap_to_leader = s["cumul_time"] - leader_time
            pos_change = prev_positions[s["driver_id"]] - s["position"]
            race_log.append(
                {
                    "lap": lap,
                    "position": s["position"],
                    "driver_id": s["driver_id"],
                    "lap_time": last_lap["lap_time"],
                    "delta": last_lap["delta"],
                    "tyre_compound": last_lap["tyre_compound"],
                    "pitted": last_lap["pitted"],
                    "gap_to_leader": gap_to_leader,
                    "cumul_time": s["cumul_time"],
                    "overtake_attempt": last_lap["overtake_attempt"],
                    "pos_change_lap": pos_change,
                    "pos_change_total": grid_pos_map[s["driver_id"]] - s["position"],
                }
            )

    print("Final classification")
    for s in driver_states:
        total_change = grid_pos_map[s["driver_id"]] - s["position"]
        print(
            f"P{s['position']:2d} {s['driver_id']:3s} total {format_seconds(s['cumul_time'])} "
            f"(grid {grid_pos_map[s['driver_id']]:2d}, {total_change:+d})"
        )

    return pd.DataFrame(race_log)


In [22]:
# Example: build a grid from the latest session and simulate a short race
lap1_rows = df_model[df_model["lap_number"] == 1]
if lap1_rows.empty:
    raise ValueError("No lap_number == 1 rows available to build a grid.")

last_row = lap1_rows.iloc[-1]
session_key = last_row["session_key"]
session_lap1 = lap1_rows[lap1_rows["session_key"] == session_key].copy().sort_values("current_position")

grid_drivers = list(session_lap1["driver_id"])
circuit_id = last_row["circuit_id"]
total_laps = int(session_lap1["total_race_laps"].iloc[0]) if not session_lap1["total_race_laps"].isna().all() else 50

print("Circuit:", circuit_id, "Grid size:", len(grid_drivers))

race_df_sim = simulate_race(
    model,
    circuit_id=circuit_id,
    grid_drivers=grid_drivers[:12],  # keep short for demo
    total_laps=min(12, total_laps),
    year=int(last_row["year"]),
    global_strategy=[(int(total_laps * 0.4), "MEDIUM"), (int(total_laps * 0.8), "SOFT")],
    safety_car_laps=[5],
    rain_laps=[],
    pit_loss=22.0,
)

race_df_sim.head()



Circuit: austin Grid size: 20
Final classification
P 1 VER total 21:35.00 (grid  1, +0)
P 2 LEC total 21:39.33 (grid  2, +0)
P 3 HAM total 21:47.67 (grid  4, +1)
P 4 SAI total 21:54.71 (grid  9, +5)
P 5 PIA total 21:55.01 (grid  5, +0)
P 6 TSU total 21:57.32 (grid 10, +4)
P 7 ANT total 22:00.36 (grid  7, +0)
P 8 BEA total 22:04.84 (grid  8, +0)
P 9 ALO total 22:05.85 (grid 12, +3)
P10 NOR total 22:06.86 (grid  3, -7)
P11 HUL total 22:07.40 (grid 11, +0)
P12 RUS total 22:07.70 (grid  6, -6)


,lap,position,driver_id,lap_time,delta,tyre_compound,pitted,gap_to_leader,cumul_time,overtake_attempt,pos_change_lap,pos_change_total
0,1,1,VER,98.576410,-2.940590,SOFT,False,0.000000,98.576410,False,0,0
1,1,2,LEC,99.519795,-1.997205,SOFT,False,1.243385,99.819795,True,0,0
2,1,3,NOR,99.519795,-1.997205,SOFT,False,1.543385,100.119795,True,0,0
3,1,4,HAM,99.519795,-1.997205,SOFT,False,1.843385,100.419795,True,0,0
4,1,5,PIA,99.519795,-1.997205,SOFT,False,2.143385,100.719795,True,0,0
